<center>
# N(utrient)-P(hytoplankton)-Z(ooplankton) Model
<center>
### A toy interactive model of ocean ecosystem dynamics

<center>
Riley X. Brady

<center>
riley.brady@colorado.edu


---
## Summary
We can create a reduced model of a complex lower-trophic ocean ecosystem with a few differential equations. Here, we choose to model three variables: an arbitrary nutrient concentration (generally thought of as a macronutrient such as nitrate), a phytoplankton concentration (maybe a dinoflagellate, since we aren't implicitly modeling the nutrients required for shell-building organisms), and a zooplankton concentration (let's think of these as little copepods).

In reality, we are just modeling a finite reservoir of nitrate, and considering how it gets redistributed around the ecosystem, given a few initial conditions and parameter settings. In other words, we aren't explicitly modeling phytoplankton cell count or biomass, but rather tracking where the nitrate goes as it is incorporated into organic matter via photosynthesis, or consumed by zooplankton.

Differential equations (DE's) are complex things to deal with. In a model like this, we have three DE's interacting with one another, because the rate of change of the given population (nutrient, phytoplankton, or zooplankton) is dependent on the current state of the other two populations. Thus, it is a lot easier to discretize a model into time steps, and reduce our DE's into algebraic equations that may be solved in reference to the current state of the system.

Here, I use an explicit time-differencing scheme (forward Euler method) to model this simple ocean ecosystem.

---

## Differential Equations Contributing to the Model
The three raw DE's are as follows (where N is our nutrients, P is our phytoplankton, and Z is our zooplankton):

$\frac{dN}{dt} = -V_{m}\left(\frac{N}{K_{N}+N}\right)f(I_{0})P+(1-\gamma)R_{m}\left(1-e^{-\lambda P}\right)Z+\epsilon P+gZ$

$\frac{dP}{dt} = V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P-R_{m}\left(1 - e^{-\lambda P}\right)Z-\epsilon P$

$\frac{dZ}{dt} = \gamma R_{m}\left(1-e^{-\lambda P}\right)Z-gZ$

---

### Terms
#### Bulk Terms
If you look closely at each DE, you note that these are simply source (+) minus sink (-) equations. This simple model only has a few nitrogen exchange processes: 

1. $V_{m}\left(\frac{N}{K_{N} + N}\right)f(I_{0})P$ : Phytoplankton grazing term. How much inorganic nitrogen are they taking up?

1. $\gamma R_{m}\left(1-e^{-\lambda P}\right)Z$ : Zooplankton grazing term. How much nitrogen are they taking up after consuming phytoplankton and releasing some as waste? On the nitrogen rate of change term, we see it accounts for a source from this inefficiency.

1. $\epsilon P$ : How much nitrogen is being returned to the pool from phytoplankton death?

1. $gZ$ : How much nitrogen is being returned to the pool from zooplankton death?

#### Phytoplankton Terms
$V_{m}$ : Maximum growth rate of an individual plankter (div per day).

$K_{N}$ : Half-saturation constant for nitrogen uptake ($\mu molNl^{-1}$). This is the nitrogen concentration at which the phytoplankton growth rate is at half its maximum value.

$f_{0}$ : Light intensity (0 to 1 weighting function). This is a simple parameterization of a more complex hyperbolic term that uses a similar term to $K_{N}$.

$\epsilon$ : Phytoplankton death rate (cells per day).

#### Zooplankton Terms
$R_{m}$ : Maximum grazing rate of zooplankton on phytoplankton (cells per day).

$\lambda$ : Grazing constant ($\mu molNl^{-1}$).

$\gamma$ : Proportion of assimilated nitrogen by zooplankton. In other words, when they graze upon a phytoplankter, how efficient are they at taking up the nitrogen? (dimensionless)

$g$ : Zooplankton death rate (critters per day).

---

### Tools
This model was built using Python 3 and visualized using [Bokeh](http://bokeh.pydata.org/en/latest/).


In [226]:
# Outside packages
import numpy as np

# Bokeh packages
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, FixedTicker
from bokeh.models.widgets import Slider
from bokeh.plotting import figure
from bokeh.charts import Area, Bar

In [227]:
# Allow Bokeh to be utilized inline with Jupyter.
output_notebook()

Loading BokehJS ...

# Default Model View

---

We first need to compute the model with some basic parameterizations that look alright. The user can then use interactivity to modify the model settings.

In [228]:
# + + + Set Up Default View of Model + + +
# Fixed Parameters
Vm        = 1    # Maximum growth rate (per day)
Kn        = 1    # Half-saturation constant for nitrogen uptake (umolN per l)
Rm        = 1    # Maximum grazing rate (per day)
g         = 0.2  # Zooplankton death rate (per day)
lambda_Z  = 0.2  # Grazing constant (umolN per l)
epsilon   = 0.1  # Phyto death rate (per day)
gamma_Z   = 0.7  # Dimensionless proportion of assimilated nitrogen by Zooplankton
f         = 0.25 # Light intensity (assumed constant)
dt        = 1    # Timestep of 1 day

# Initial Conditions
N_0 = 4
P_0 = 2.5
Z_0 = 0.5

# Initialize Arrays
N = np.empty(100, dtype="float")
P = np.empty(100, dtype="float")
Z = np.empty(100, dtype="float")

# Insert Initial Values
N[0] = N_0
P[0] = P_0
Z[0] = Z_0

In [229]:
# + + + Run Default model + + +
# Here we use the Euler forward method to solve for t+1 and reference t. 
# This is why we need initial conditions, so we have something to reference outright.
for idx in np.arange(1, 100, 1):
    t = idx - 1
    
    # Common terms
    gamma_N   = N[t] / (Kn + N[t])
    zoo_graze = Rm * (1 - np.exp(-lambda_Z * P[t])) * Z[t]
    
    # Equation calculations
    N[idx] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
    P[idx] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
    Z[idx] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t]   

In [230]:
# Initial Datapoints
x = np.arange(1, 101, 1)
N = N
P = P
Z = Z

# Bokeh likes reading data via its own version of dictionaries.
# I also prime this dictionary with additional variables for 
# multiple plots, which makes the custom JS interaction a lot easier to manage.
source = ColumnDataSource(data = {
        'x'    : x,
        'N'    : N,
        'P'    : P,
        'Z'    : Z,
        'Psum' : N + P,
        'Zsum' : N + P + Z,
    })

In [231]:
# Functions for plotting
def plotlines(plot, x, y, source, legend, line_width=2, line_alpha=0.6,
             color='black'):
    plot.line(x, y, source=source, line_width=line_width,
             line_alpha=line_alpha, color=color, legend=legend)

# Visualization

---

To start, I build a Bokeh object that serves as the default view prior to manipulation. I comment throughout, but don't want to go through the nitty-gritty. Bokeh visualization details can be found in their user guide. 

The key to this code block is the CustomJS feature, where I allow for callbacks from the sliders and interactive bits on the figure.

In [232]:
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# TIME SERIES
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
plot = figure(plot_width=900, plot_height=300,
             toolbar_location="right", tools = "save,ywheel_zoom",
             x_range=(0, 100))

# Plot data
plotlines(plot, 'x', 'N', source, "Nutrients", color="navy")
plotlines(plot, 'x', 'P', source, "Phytoplankton", color="green")
plotlines(plot, 'x', 'Z', source, "Zooplankton", color="black")

# Plot aesthetics
plot.title.text       = 'Simple NPZ Model'
plot.yaxis.axis_label = 'Concentration (umolN per L)'
plot.xaxis.axis_label = 'Model Days'
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# STACKED BAR PLOT
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# Create figure
plot2 = figure(plot_width=900, plot_height=300,
            toolbar_location="right", tools = "save,ywheel_zoom",
             x_range=(0, 100), y_range=plot.y_range)

# Plot data
plot2.vbar('x', width=0.2, bottom=0, top='N', source=source, color="blue")
plot2.vbar('x', width=0.2, bottom='N', top='Psum', source=source, color="green")
plot2.vbar('x', width=0.2, bottom='Psum', top='Zsum', source=source, color="black")

# Aesthetics
plot2.title.text       = 'Nutrient Distribution'
plot2.yaxis.axis_label = 'Concentration (umolN per L)'
plot2.xaxis.axis_label = 'Model Days'
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 


# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
# JAVASCRIPT INTERACTION
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 
callback = CustomJS(args=dict(source=source), code="""
    // Ingest main model data for modification
    var data = source.get('data');
    x    = data['x'];
    N    = data['N'];
    P    = data['P'];
    Z    = data['Z'];
    Psum = data['Psum'];
    Zsum = data['Zsum'];
    
    // Parameters
    var Vm = 1;
    var Kn = 1;
    var Rm = 1;
    var g = 0.2;
    var lambda_Z = 0.2;
    var epsilon = 0.1;
    var gamma_Z = 0.7;
    var f = 0.25;
    var dt = 1;
    
    // Initial Conditions with modifications allowed
    var N_0 = nut.get('value');
    var P_0 = phyto.get('value');
    var Z_0 = zoo.get('value');
    
    // Insert Initial Values for model
    N[0]    = N_0;
    P[0]    = P_0;
    Z[0]    = Z_0;
    Psum[0] = N_0 + P_0;
    Zsum[0] = N_0 + P_0 + Z_0;
    
    // Run Model
    for (i = 1; i < x.length; i++) {
         t = i - 1;

        // Common terms
        gamma_N   = N[t] / (Kn + N[t])
        zoo_graze = Rm * (1 - Math.exp(-lambda_Z * P[t])) * Z[t]
    
        // Equation calculations for model
        N[i] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
        P[i] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
        Z[i] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t];
        
        // Sum Variables
        Psum[i] = N[i] + P[i];
        Zsum[i] = N[i] + P[i] + Z[i];
    }
    source.trigger('change');
""")

# With multiple widgets, need to set a name for when calling them.

# Nutrient
nut_slider = Slider(start = 0, end = 10, value = 4, step = 0.1, title = "Initial Nutrient Concentration", callback=callback)
callback.args["nut"] = nut_slider

# Phyto
phyto_slider = Slider(start = 0, end = 10, value = 2.5, step = 0.1, title = "Initial Phytoplankton Concentration", callback=callback)
callback.args["phyto"] = phyto_slider

# Zoo
zoo_slider = Slider(start = 0, end = 10, value = 0.5, step = 0.1, title = "Initial Zooplankton Concentration", callback=callback)
callback.args["zoo"] = zoo_slider
# + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

# Set up layout
layout = column(nut_slider, phyto_slider, zoo_slider, plot, plot2)

show(layout)